In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #1 input image chanel, 6 output channel, 3x3 square convolution
        #kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        #operation
        self.fc1 = nn.Linear(16*6*6, 120) #6*6 from image dimention
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #Maxpooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #if the size is a square 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] #all dimention except batch size
        num_features = 1
        for s in size:
            num_features*=s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
#caculate num of parameter
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [3]:
input = torch.rand(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0667,  0.0409,  0.0115,  0.0347,  0.0788, -0.0179, -0.0647,  0.0073,
          0.0646, -0.0578]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
out.backward(torch.rand(1, 10))

In [6]:
#loss function

output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterition = nn.MSELoss()

loss = criterition(output, target)
print(loss)

tensor(1.0645, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])


BackProp

In [8]:
net.zero_grad() #zero gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0136,  0.0014,  0.0004, -0.0049, -0.0018,  0.0035])


In [10]:
learning_rate = 0.001
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)


In [12]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad() #zeros gradient buffer
output = net(input)
loss = criterition(output, target)
loss.backward()
optimizer.step()